In [1]:
# Imports
import numpy as np 
import random
import networkx as nx
from networkx import karate_club_graph, to_numpy_matrix

In [2]:
# Convolution Layer
G= karate_club_graph()
order = sorted(list(G.nodes()))
#nx.draw(G, with_labels=True)
A = nx.to_numpy_matrix(G, nodelist=order)
I = np.eye(G.number_of_nodes())
A_own = A + I
D_own = np.array(np.sum(A_own, axis=0))[0]
D_inv=D_own**-0.5
D_inv = np.matrix(np.diag(D_inv))
W1 = np.random.normal(loc=0, scale=1, size=(G.number_of_nodes(), 6))
W2 = np.random.normal(loc=0, size=(W1.shape[1], 3))
def gcn_layer(A, D, X, W):
    feature=D* A * D * X * W
    feature=np.maximum(0,feature)
    return feature
H1 = gcn_layer(A_own, D_inv, I, W1)
H2 = gcn_layer(A_own, D_inv, H1, W2)
#H2

In [3]:
#Community Detection
# remove edge to construct subgraphs
def edge_to_remove(G):
    dict1=nx.edge_betweenness_centrality(G)
    list_of_tuples=dict1.items()
    list_of_tuples=sorted(list_of_tuples,key=lambda X:X[1],reverse=True)
    return list_of_tuples[0][0]
#Girvan Algorithm
def girvan(G):
    c=list(nx.connected_component_subgraphs(G))
    l=len(c)
    print("the number of components",l)
    while(l==1):
        G.remove_edge(*edge_to_remove(G))
        c=list(nx.connected_component_subgraphs(G))
        l=len(c)
        #print("the number of components",l)
    return c
G= nx.karate_club_graph()
#print("Main Graph")
#nx.draw(G, with_labels=True)
c=girvan(G)
#print("Communities")
nx.draw(G, with_labels=True)
#preperation of class label
y=np.zeros((G.number_of_nodes(),1))
#print("------------")
k=0
for i in c:
    g1=list(i.nodes())
    for j in range(i.number_of_nodes()):
        y[g1[j]]=k
    #print("The Number of Nodes : ",i.number_of_nodes())
    #print(i.nodes())
    #print("---------")
    k=k+1
#print("Labels of class : ",y)

the number of components 1


C:\ProgramData\Anaconda3\lib\site-packages\networkx\drawing\nx_pylab.py:579: MatplotlibDeprecationWarning: 
The iterable function was deprecated in Matplotlib 3.1 and will be removed in 3.3. Use np.iterable instead.
  if not cb.iterable(width):


In [4]:
# Fully Connected Neural Network

# Class definition
class FullyConnectedNeuralNetwork:
    def __init__(self, x,y):
        self.input = np.array(x)
        self.weights1= np.random.rand(self.input.shape[1],4) # considering we have 4 nodes in the hidden layer
        self.weights2 = np.random.rand(4,1)
        self.y = y
        self.output = np.zeros(y.shape)
    # Activation function
    def sigmoid(self, x):
        return 1/(1+np.exp(-x))
    # Derivative of sigmoid
    def sigmoid_derivative(self, x):
        return (x * (1 - x))  
    #Feed Forward
    def feedforward(self):
        self.layer1 = self.sigmoid(np.dot(self.input, self.weights1))
        self.layer2 = self.sigmoid(np.dot(self.layer1, self.weights2))
        return self.layer2
    #Backpropagation 
    def backprop(self):
        d_weights2 = np.dot(self.layer1.T, 2*(self.y -self.output)*self.sigmoid_derivative(self.output))
        d_weights1 = np.dot(self.input.T, np.dot(2*(self.y -self.output)*
                     self.sigmoid_derivative(self.output), self.weights2.T)*self.sigmoid_derivative(self.layer1))
        self.weights1 += d_weights1
        self.weights2 += d_weights2
    #Train
    def classfication(self):
        fo=np.zeros(self.output.shape)
        j=0
        for i in self.output:
            if i>=0.5:
                fo[j]=1
            else:
                fo[j]=0
            j=j+1
        return fo
    def train(self):
        for i in range(100000):
            self.output = self.feedforward()
            self.backprop()
        finaloutput=self.classfication()
        #for i in finaloutput:
        print("Weights1:")
        print(self.weights1)
        print("-------------")
        print("Weights2:")
        print(self.weights2)
        print("-------------")
        print("Predictive Output:")
        print(self.output.T)
        print("Predictive Output class:")
        print(finaloutput.T)
        print("Actual Output:")
        print(self.y.T)        

In [5]:
nn=FullyConnectedNeuralNetwork(H2,y)
nn.train()

Weights1:
[[ 4.03066301 -5.35693779 -1.58832296 -0.63250339]
 [ 0.80940955  0.96125754  0.1026228   0.76505702]
 [-4.75670582  2.06918768 -9.59686294 -4.77832637]]
-------------
Weights2:
[[-11.04609985]
 [ 10.32210098]
 [ 10.30431665]
 [-10.82360771]]
-------------
Predictive Output:
[[4.99651227e-01 7.75571880e-01 6.04342600e-01 4.99651227e-01
  3.53130216e-04 1.82777444e-04 1.23209235e-05 4.99651227e-01
  8.37877545e-01 4.99651227e-01 4.64100288e-04 8.26566644e-01
  4.99651227e-01 5.81449662e-01 5.52347451e-01 7.26795611e-01
  2.24517948e-05 5.11460675e-01 5.51588206e-01 7.19383204e-01
  5.90466976e-01 6.97181926e-01 5.52347451e-01 7.94105350e-01
  9.18008492e-01 8.67858786e-01 9.14850258e-01 5.49693104e-01
  8.54243490e-01 8.01958875e-01 9.31256203e-01 9.30870101e-01
  9.54424397e-01 9.73212907e-01]]
Predictive Output class:
[[0. 1. 1. 0. 0. 0. 0. 0. 1. 0. 0. 1. 0. 1. 1. 1. 0. 1. 1. 1. 1. 1. 1. 1.
  1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]]
Actual Output:
[[0. 0. 1. 0. 0. 0. 0. 0. 1. 1. 0. 0